<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/eurusd_automl_hist_range.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flaml

     |████████████████████████████████| 137 kB 5.4 MB/s 
     |████████████████████████████████| 2.0 MB 36.5 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [2]:
#designed for Jupyter/kaggle/colab
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor as rf
#import auto hyperparameter tuning
from flaml import AutoML
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor


In [3]:
 #MT4 csv 
df1=pd.read_csv('EURUSD1440.csv', names=['d','t','eu_o','eu_h','eu_l','eu_c','eu_v'])
df_m_5 = df1.tail(6000)
df2 = df1.tail(6000)
# df2=pd.read_csv('GBPUSD1440.csv', names=['d','t','gu_o','gu_h','gu_l','gu_c','gu_v'])
# df2 = df2.tail(2000)

# df3=pd.read_csv('USDCAD1440.csv', names=['d','t','uc_o','uc_h','uc_l','uc_c','uc_v'])
# df3 = df3.tail(2000)

# df4=pd.read_csv('USDCHF1440.csv', names=['d','t','uf_o','uf_h','uf_l','uf_c','uf_v'])
# df4 = df4.tail(2000)

# df5=pd.read_csv('USDJPY1440.csv', names=['d','t','uj_o','uj_h','uj_l','uj_c','uj_v'])
# df5 = df5.tail(2000)

# df6=pd.read_csv('USDSEK1440.csv', names=['d','t','us_o','us_h','us_l','us_c','us_v'])
# df6 = df6.tail(2000)

# #using merge
# df_m_1 = df1.merge(df2, on='d')
# df_m_2 = df_m_1.merge(df3, on='d')
# df_m_3 = df_m_2.merge(df4, on='d')
# df_m_4 = df_m_3.merge(df5, on='d')
# df_m_5 = df_m_4.merge(df6, on='d')
# df_m_5 = df_m_5.drop(columns='t_x')
# df_m_5 = df_m_5.drop(columns='t_y')

# pd.set_option('display.max_columns', None)
# print(df_m_5) 

In [4]:
df2.tail()

,d,t,eu_o,eu_h,eu_l,eu_c,eu_v
12788,2021.12.10,00:00,1.12913,1.13239,1.12650,1.13165,76961
12789,2021.12.13,00:00,1.13106,1.13191,1.12599,1.12854,67794
12790,2021.12.14,00:00,1.12828,1.13236,1.12535,1.12577,69981
12791,2021.12.15,00:00,1.12549,1.12990,1.12215,1.12936,93710
12792,2021.12.16,00:00,1.12865,1.12987,1.12845,1.12929,1165


In [5]:

# #2 days forward
# df_m_5['eu_High_next_day2'] = df_m_5['eu_h'].shift(-2)
# df_m_5['eu_Low_next_day2'] = df_m_5['eu_l'].shift(-2)
# df_m_5['eu_Close_next_day2'] = df_m_5['eu_c'].shift(-2) 
# #3 days forward
# df_m_5['eu_High_next_day3'] = df_m_5['eu_h'].shift(-3)
# df_m_5['eu_Low_next_day3'] = df_m_5['eu_l'].shift(-3)
# df_m_5['eu_Close_next_day3'] = df_m_5['eu_c'].shift(-3) 
# #4 days forward 
# df_m_5['eu_High_next_day4'] = df_m_5['eu_h'].shift(-4)
# df_m_5['eu_Low_next_day4'] = df_m_5['eu_l'].shift(-4)
# df_m_5['eu_Close_next_day4'] = df_m_5['eu_c'].shift(-4) 
# #5 days forward
# df_m_5['eu_High_next_day5'] = df_m_5['eu_h'].shift(-5)
# df_m_5['eu_Low_next_day5'] = df_m_5['eu_l'].shift(-5)
# df_m_5['eu_Close_next_day5'] = df_m_5['eu_c'].shift(-5) 

#Features
#1 day ago (features)
df_m_5['eu_High_last_day1'] = df_m_5['eu_h'].shift(1)
df_m_5['eu_Low_last_day1'] = df_m_5['eu_l'].shift(1)
df_m_5['eu_Close_last_day1'] = df_m_5['eu_c'].shift(1) 
#2 days ago
df_m_5['eu_High_last_day2'] = df_m_5['eu_h'].shift(2)
df_m_5['eu_Low_last_day2'] = df_m_5['eu_l'].shift(2)
df_m_5['eu_Close_last_day2'] = df_m_5['eu_c'].shift(2) 
#3 days ago
df_m_5['eu_High_last_day3'] = df_m_5['eu_h'].shift(3)
df_m_5['eu_Low_last_day3'] = df_m_5['eu_l'].shift(3)
df_m_5['eu_Close_last_day3'] = df_m_5['eu_c'].shift(3) 
#4 days ago 
df_m_5['eu_High_last_day4'] = df_m_5['eu_h'].shift(4)
df_m_5['eu_Low_last_day4'] = df_m_5['eu_l'].shift(4)
df_m_5['eu_Close_last_day4'] = df_m_5['eu_c'].shift(4) 
#5 days ago
df_m_5['eu_High_last_day5'] = df_m_5['eu_h'].shift(5)
df_m_5['eu_Low_last_day5'] = df_m_5['eu_l'].shift(5)
df_m_5['eu_Close_last_day5'] = df_m_5['eu_c'].shift(5) 
#6 day ago (features)
df_m_5['eu_High_last_day6'] = df_m_5['eu_h'].shift(6)
df_m_5['eu_Low_last_day6'] = df_m_5['eu_l'].shift(6)
df_m_5['eu_Close_last_day6'] = df_m_5['eu_c'].shift(6) 
#7 days ago
df_m_5['eu_High_last_day7'] = df_m_5['eu_h'].shift(7)
df_m_5['eu_Low_last_day7'] = df_m_5['eu_l'].shift(7)
df_m_5['eu_Close_last_day7'] = df_m_5['eu_c'].shift(7) 
#8 days ago
df_m_5['eu_High_last_day8'] = df_m_5['eu_h'].shift(8)
df_m_5['eu_Low_last_day8'] = df_m_5['eu_l'].shift(8)
df_m_5['eu_Close_last_day8'] = df_m_5['eu_c'].shift(8) 
#9 days ago 
df_m_5['eu_High_last_day9'] = df_m_5['eu_h'].shift(9)
df_m_5['eu_Low_last_day9'] = df_m_5['eu_l'].shift(9)
df_m_5['eu_Close_last_day9'] = df_m_5['eu_c'].shift(9) 
#10 days ago
df_m_5['eu_High_last_day10'] = df_m_5['eu_h'].shift(10)
df_m_5['eu_Low_last_day10'] = df_m_5['eu_l'].shift(10)
df_m_5['eu_Close_last_day10'] = df_m_5['eu_c'].shift(10) 

#Target
#1 day forward 
df_m_5['eu_High_next_day1'] = df_m_5['eu_h'].shift(-1)
df_m_5['eu_Low_next_day1'] = df_m_5['eu_l'].shift(-1)
df_m_5['eu_Close_next_day1'] = df_m_5['eu_c'].shift(-1) 

#drop null and useless columns
df_m_5 = df_m_5.dropna()
df_m_5 = df_m_5.drop(['t','eu_v'], axis=1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [6]:
df_m_5 

,d,eu_o,eu_h,eu_l,eu_c,eu_High_last_day1,eu_Low_last_day1,eu_Close_last_day1,eu_High_last_day2,eu_Low_last_day2,eu_Close_last_day2,eu_High_last_day3,eu_Low_last_day3,eu_Close_last_day3,eu_High_last_day4,eu_Low_last_day4,eu_Close_last_day4,eu_High_last_day5,eu_Low_last_day5,eu_Close_last_day5,eu_High_last_day6,eu_Low_last_day6,eu_Close_last_day6,eu_High_last_day7,eu_Low_last_day7,eu_Close_last_day7,eu_High_last_day8,eu_Low_last_day8,eu_Close_last_day8,eu_High_last_day9,eu_Low_last_day9,eu_Close_last_day9,eu_High_last_day10,eu_Low_last_day10,eu_Close_last_day10,eu_High_next_day1,eu_Low_next_day1,eu_Close_next_day1
6803,1998.11.16,1.16800,1.18430,1.16800,1.17850,1.17010,1.16140,1.16830,1.17300,1.15820,1.16700,1.17410,1.16570,1.17330,1.17120,1.16170,1.16990,1.17650,1.15990,1.16480,1.18500,1.17180,1.17460,1.18700,1.17700,1.18260,1.18540,1.17310,1.17820,1.18850,1.17780,1.18280,1.19430,1.18610,1.18750,1.18600,1.17650,1.17890
6804,1998.11.17,1.17900,1.18600,1.17650,1.17890,1.18430,1.16800,1.17850,1.17010,1.16140,1.16830,1.17300,1.15820,1.16700,1.17410,1.16570,1.17330,1.17120,1.16170,1.16990,1.17650,1.15990,1.16480,1.18500,1.17180,1.17460,1.18700,1.17700,1.18260,1.18540,1.17310,1.17820,1.18850,1.17780,1.18280,1.18390,1.17250,1.17450
6805,1998.11.18,1.17900,1.18390,1.17250,1.17450,1.18600,1.17650,1.17890,1.18430,1.16800,1.17850,1.17010,1.16140,1.16830,1.17300,1.15820,1.16700,1.17410,1.16570,1.17330,1.17120,1.16170,1.16990,1.17650,1.15990,1.16480,1.18500,1.17180,1.17460,1.18700,1.17700,1.18260,1.18540,1.17310,1.17820,1.17780,1.16430,1.16610
6806,1998.11.19,1.17300,1.17780,1.16430,1.16610,1.18390,1.17250,1.17450,1.18600,1.17650,1.17890,1.18430,1.16800,1.17850,1.17010,1.16140,1.16830,1.17300,1.15820,1.16700,1.17410,1.16570,1.17330,1.17120,1.16170,1.16990,1.17650,1.15990,1.16480,1.18500,1.17180,1.17460,1.18700,1.17700,1.18260,1.17100,1.16030,1.16240
6807,1998.11.20,1.16600,1.17100,1.16030,1.16240,1.17780,1.16430,1.16610,1.18390,1.17250,1.17450,1.18600,1.17650,1.17890,1.18430,1.16800,1.17850,1.17010,1.16140,1.16830,1.17300,1.15820,1.16700,1.17410,1.16570,1.17330,1.17120,1.16170,1.16990,1.17650,1.15990,1.16480,1.18500,1.17180,1.17460,1.16350,1.14850,1.15450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12787,2021.12.09,1.13422,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13295,1.12051,1.13177,1.12295,1.11959,1.12062,1.13239,1.12650,1.13165
12788,2021.12.10,1.12913,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13295,1.12051,1.13177,1.13191,1.12599,1.12854
12789,2021.12.13,1.13106,1.13191,1.12599,1.12854,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13236,1.12535,1.12577
12790,2021.12.14,1.12828,1.13236,1.12535,1.12577,1.13191,1.12599,1.12854,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.12990,1.12215,1.12936


In [7]:
df_m_5.insert(1,'index',df_m_5.index)

In [8]:
# #new column: high vs low
# df_m_5['eu_h_or_l'] = ((df_m_5.eu_h - df_m_5.eu_o) > (df_m_5.eu_o - df_m_5.eu_l)) 
# df_m_5.eu_h_or_l = df_m_5.eu_h_or_l.replace({True:1,False:0})

# #new column: close vs yesterday
# df_m_5['eu_c_vs_c'] = (df_m_5.eu_c > df_m_5.eu_o) 
# df_m_5.eu_c_vs_c = df_m_5.eu_c_vs_c.replace({True:1,False:0})

# #new column: shift tomorrow to today
# df_m_5['eu_High_next_day'] = df_m_5['eu_h'].shift(-1)
# df_m_5['eu_Low_next_day'] = df_m_5['eu_l'].shift(-1)
# df_m_5['eu_Close_next_day'] = df_m_5['eu_c'].shift(-1)
# df_m_5 = df_m_5.dropna()


In [9]:
df_m_5.tail()

,d,index,eu_o,eu_h,eu_l,eu_c,eu_High_last_day1,eu_Low_last_day1,eu_Close_last_day1,eu_High_last_day2,eu_Low_last_day2,eu_Close_last_day2,eu_High_last_day3,eu_Low_last_day3,eu_Close_last_day3,eu_High_last_day4,eu_Low_last_day4,eu_Close_last_day4,eu_High_last_day5,eu_Low_last_day5,eu_Close_last_day5,eu_High_last_day6,eu_Low_last_day6,eu_Close_last_day6,eu_High_last_day7,eu_Low_last_day7,eu_Close_last_day7,eu_High_last_day8,eu_Low_last_day8,eu_Close_last_day8,eu_High_last_day9,eu_Low_last_day9,eu_Close_last_day9,eu_High_last_day10,eu_Low_last_day10,eu_Close_last_day10,eu_High_next_day1,eu_Low_next_day1,eu_Close_next_day1
12787,2021.12.09,12787,1.13422,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13295,1.12051,1.13177,1.12295,1.11959,1.12062,1.13239,1.12650,1.13165
12788,2021.12.10,12788,1.12913,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13295,1.12051,1.13177,1.13191,1.12599,1.12854
12789,2021.12.13,12789,1.13106,1.13191,1.12599,1.12854,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13236,1.12535,1.12577
12790,2021.12.14,12790,1.12828,1.13236,1.12535,1.12577,1.13191,1.12599,1.12854,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.12990,1.12215,1.12936
12791,2021.12.15,12791,1.12549,1.12990,1.12215,1.12936,1.13236,1.12535,1.12577,1.13191,1.12599,1.12854,1.13239,1.12650,1.13165,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.12987,1.12845,1.12929


In [10]:

df_m_5.isna().sum()

d                      0
index                  0
eu_o                   0
eu_h                   0
eu_l                   0
eu_c                   0
eu_High_last_day1      0
eu_Low_last_day1       0
eu_Close_last_day1     0
eu_High_last_day2      0
eu_Low_last_day2       0
eu_Close_last_day2     0
eu_High_last_day3      0
eu_Low_last_day3       0
eu_Close_last_day3     0
eu_High_last_day4      0
eu_Low_last_day4       0
eu_Close_last_day4     0
eu_High_last_day5      0
eu_Low_last_day5       0
eu_Close_last_day5     0
eu_High_last_day6      0
eu_Low_last_day6       0
eu_Close_last_day6     0
eu_High_last_day7      0
eu_Low_last_day7       0
eu_Close_last_day7     0
eu_High_last_day8      0
eu_Low_last_day8       0
eu_Close_last_day8     0
eu_High_last_day9      0
eu_Low_last_day9       0
eu_Close_last_day9     0
eu_High_last_day10     0
eu_Low_last_day10      0
eu_Close_last_day10    0
eu_High_next_day1      0
eu_Low_next_day1       0
eu_Close_next_day1     0
dtype: int64

In [11]:
df_m_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5989 entries, 6803 to 12791
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   d                    5989 non-null   object 
 1   index                5989 non-null   int64  
 2   eu_o                 5989 non-null   float64
 3   eu_h                 5989 non-null   float64
 4   eu_l                 5989 non-null   float64
 5   eu_c                 5989 non-null   float64
 6   eu_High_last_day1    5989 non-null   float64
 7   eu_Low_last_day1     5989 non-null   float64
 8   eu_Close_last_day1   5989 non-null   float64
 9   eu_High_last_day2    5989 non-null   float64
 10  eu_Low_last_day2     5989 non-null   float64
 11  eu_Close_last_day2   5989 non-null   float64
 12  eu_High_last_day3    5989 non-null   float64
 13  eu_Low_last_day3     5989 non-null   float64
 14  eu_Close_last_day3   5989 non-null   float64
 15  eu_High_last_day4    5989 non-null

In [12]:
#extract row for variable
last_row = df_m_5.iloc[-1:,np.r_[2:36]].values
print(last_row) 

[[1.12549 1.1299  1.12215 1.12936 1.13236 1.12535 1.12577 1.13191 1.12599
  1.12854 1.13239 1.1265  1.13165 1.13452 1.12782 1.12924 1.13547 1.12627
  1.13418 1.12982 1.12273 1.12681 1.13109 1.12664 1.12848 1.13333 1.12665
  1.13088 1.13475 1.12952 1.13013 1.13594 1.13023 1.1319 ]]


In [33]:
#for high prediction
#select rows to use for x,y
x_h = df_m_5.iloc[:,np.r_[2:36]].values
print(x_h)
y_h = df_m_5.iloc[:,[36]].values
print(y_h) 

[[1.168   1.1843  1.168   ... 1.1943  1.1861  1.1875 ]
 [1.179   1.186   1.1765  ... 1.1885  1.1778  1.1828 ]
 [1.179   1.1839  1.1725  ... 1.1854  1.1731  1.1782 ]
 ...
 [1.13106 1.13191 1.12599 ... 1.13117 1.12582 1.12918]
 [1.12828 1.13236 1.12535 ... 1.13827 1.1235  1.1339 ]
 [1.12549 1.1299  1.12215 ... 1.13594 1.13023 1.1319 ]]
[[1.186  ]
 [1.1839 ]
 [1.1778 ]
 ...
 [1.13236]
 [1.1299 ]
 [1.12987]]


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_h, y_h, test_size = 0.30)

In [15]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto", n_jobs=-1)

[flaml.automl: 12-15 23:48:28] {1943} INFO - task = regression
[flaml.automl: 12-15 23:48:28] {1945} INFO - Data split method: uniform
[flaml.automl: 12-15 23:48:28] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-15 23:48:28] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-15 23:48:28] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-15 23:48:28] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-15 23:48:29] {2425} INFO - Estimated sufficient time budget=6784s. Estimated necessary time budget=48s.
[flaml.automl: 12-15 23:48:29] {2505} INFO -  at 0.7s,	estimator lgbm's best error=0.4764,	best estimator lgbm's best error=0.4764
[flaml.automl: 12-15 23:48:29] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-15 23:48:29] {2505} INFO -  at 0.9s,	estimator lgbm's best error=0.4764,	best estimator lgbm's best error=0.4764
[flaml.automl: 12-15 23:48:29] {2311} INFO 

In [35]:
#paste retrained model
sugg_reg_high_param = LGBMRegressor(colsample_bytree=0.7463308378914483,
              learning_rate=0.1530612501227463, max_bin=1023,
              min_child_samples=2, n_estimators=49, num_leaves=42,
              reg_alpha=0.0009765625, reg_lambda=0.012698515198279517,
              verbose=-1)
#paste finished
sugg_reg_high_param.fit(X_train,y_train)
y_pred_h=sugg_reg_high_param.predict(last_row)
y_pred_h

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1.13297088])

In [17]:
y_pred = sugg_reg_high_param.predict(X_test)

In [18]:
def mean_absolute_percentage_error(y_true,y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test,y_pred)

In [19]:
#for regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.003927110963830078
MSE: 2.9135815026710365e-05
RMSE: 0.005397760186105934
MAPE: 15.464261710789163
MedAE 0.002932238682283206


In [36]:
#for low prediction
#select rows to use for x,y
x_l = df_m_5.iloc[:,np.r_[2:36]].values
print(x_l)
y_l = df_m_5.iloc[:,[37]].values
print(y_l) 

[[1.168   1.1843  1.168   ... 1.1943  1.1861  1.1875 ]
 [1.179   1.186   1.1765  ... 1.1885  1.1778  1.1828 ]
 [1.179   1.1839  1.1725  ... 1.1854  1.1731  1.1782 ]
 ...
 [1.13106 1.13191 1.12599 ... 1.13117 1.12582 1.12918]
 [1.12828 1.13236 1.12535 ... 1.13827 1.1235  1.1339 ]
 [1.12549 1.1299  1.12215 ... 1.13594 1.13023 1.1319 ]]
[[1.1765 ]
 [1.1725 ]
 [1.1643 ]
 ...
 [1.12535]
 [1.12215]
 [1.12845]]


In [37]:
#for low prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_l, y_l, test_size = 0.30)

In [22]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto", n_jobs=-1)

[flaml.automl: 12-15 23:49:30] {1943} INFO - task = regression
[flaml.automl: 12-15 23:49:30] {1945} INFO - Data split method: uniform
[flaml.automl: 12-15 23:49:30] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-15 23:49:30] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-15 23:49:30] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-15 23:49:30] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-15 23:49:30] {2425} INFO - Estimated sufficient time budget=1582s. Estimated necessary time budget=11s.
[flaml.automl: 12-15 23:49:30] {2505} INFO -  at 0.2s,	estimator lgbm's best error=0.4762,	best estimator lgbm's best error=0.4762
[flaml.automl: 12-15 23:49:30] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-15 23:49:30] {2505} INFO -  at 0.4s,	estimator lgbm's best error=0.4762,	best estimator lgbm's best error=0.4762
[flaml.automl: 12-15 23:49:30] {2311} INFO 

In [38]:
#paste retrained model
sugg_reg_low_param = LGBMRegressor(colsample_bytree=0.7463308378914483,
              learning_rate=0.1530612501227463, max_bin=1023,
              min_child_samples=2, n_estimators=49, num_leaves=42,
              reg_alpha=0.0009765625, reg_lambda=0.012698515198279517,
              verbose=-1)
#paste finished
sugg_reg_low_param.fit(X_train,y_train)
y_pred_l=sugg_reg_low_param.predict(last_row)
y_pred_l

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1.12580923])

In [24]:
#for low pred 
y_pred = sugg_reg_low_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [25]:
#for low pred regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.0037634512832398943
MSE: 2.7494512118065878e-05
RMSE: 0.0052435209657315076
MAPE: 15.388373519886272
MedAE 0.002822901271649858


In [39]:
#for close prediction
#select rows to use for x,y
x_c = df_m_5.iloc[:,np.r_[2:36]].values
print(x_c)
y_c = df_m_5.iloc[:,[38]].values
print(y_c) 

[[1.168   1.1843  1.168   ... 1.1943  1.1861  1.1875 ]
 [1.179   1.186   1.1765  ... 1.1885  1.1778  1.1828 ]
 [1.179   1.1839  1.1725  ... 1.1854  1.1731  1.1782 ]
 ...
 [1.13106 1.13191 1.12599 ... 1.13117 1.12582 1.12918]
 [1.12828 1.13236 1.12535 ... 1.13827 1.1235  1.1339 ]
 [1.12549 1.1299  1.12215 ... 1.13594 1.13023 1.1319 ]]
[[1.1789 ]
 [1.1745 ]
 [1.1661 ]
 ...
 [1.12577]
 [1.12936]
 [1.12929]]


In [40]:
#for close prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_c, y_c, test_size = 0.30)

In [28]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto")

[flaml.automl: 12-15 23:50:31] {1943} INFO - task = regression
[flaml.automl: 12-15 23:50:31] {1945} INFO - Data split method: uniform
[flaml.automl: 12-15 23:50:31] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-15 23:50:31] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-15 23:50:31] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-15 23:50:31] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-15 23:50:31] {2425} INFO - Estimated sufficient time budget=1693s. Estimated necessary time budget=12s.
[flaml.automl: 12-15 23:50:31] {2505} INFO -  at 0.2s,	estimator lgbm's best error=0.4770,	best estimator lgbm's best error=0.4770
[flaml.automl: 12-15 23:50:31] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-15 23:50:32] {2505} INFO -  at 0.4s,	estimator lgbm's best error=0.4770,	best estimator lgbm's best error=0.4770
[flaml.automl: 12-15 23:50:32] {2311} INFO 

In [41]:
#paste retrained model
sugg_reg_close_param = RandomForestRegressor(max_features=0.5946562283064879, max_leaf_nodes=69,
                      n_estimators=37, n_jobs=-1)
#paste finished
sugg_reg_close_param.fit(X_train,y_train)
y_pred_c=sugg_reg_close_param.predict(last_row)
y_pred_c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


array([1.12965861])

In [30]:
#for close prediction
y_pred = sugg_reg_close_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [31]:
#for close regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 
#print('MdAPE', metrics)

MAE: 0.005715509386521925
MSE: 6.030726220075883e-05
RMSE: 0.007765775054735929
MAPE: 15.350456226699775
MedAE 0.00429101422858702


In [32]:
print('Today the predictions are:')
print('high', y_pred_h)
print('close', y_pred_c)
print('low', y_pred_l)
print('Best used as polarities.')

Today the predictions are:
high [1.13239197]
close [1.12890832]
low [1.12493266]
Best used as polarities.
